# Community Detection

In [1]:
import networkx as nx
import pandas as pd
import numpy as np

from node2vec import Node2Vec as n2v

In [2]:
# constants


## Import Data

## Create Network

## Algorithmic Community Detection  
There are many algorithmic algorithms associated with community detection on networks. In this tutorial I will cover the Greedy and Girvan Newman's approach to community detection. There are many other algorithms (like Louvaine's algorithm) which does community detection on networks and I highly encourage you to research and implement those as well. You can find a lot of them here in the NetworkX [documentation](https://networkx.org/documentation/stable/reference/algorithms/community.html) [1]. I will go over the intution and python implementation of these two algorithms and see how they differ on the network in comparison to a semi-supervised machine learning approach. 

### Girvan Newman's Algorithm

### Greedy Community Detection

## Apply Node2Vec

## Generate Embeddings

## k-Means

## Model Performance

## Concluding Remarks

## Resources
- [1] https://networkx.org/documentation/stable/reference/algorithms/community.html

---